# 1. 基础设置

In [13]:
# 导包
import torch
import os
import argparse
import pandas as pd
import tqdm
import numpy as np
# os.environ['NUMEXPR_MAX_THREADS'] = '48'

# 导入各个指标
import sys
sys.path.append('/home/dengruijun/data/FinTech/PP-Split/')
from ppsplit.quantification.distance_correlation.distCor import distCorMetric
from ppsplit.quantification.fisher_information.dFIL_inverse import dFILInverseMetric
from ppsplit.quantification.shannon_information.mutual_information import MuInfoMetric
from ppsplit.quantification.shannon_information.ULoss import ULossMetric
from ppsplit.quantification.rep_reading.rep_reader import PCA_Reader
from ppsplit.quantification.shannon_information.ITE_tools import Shannon_quantity

from target_model.task_select import get_dataloader_and_model,get_dataloader_and_model, get_dataloader,get_models

# utils
from ppsplit.utils.utils import create_dir

In [26]:
args = {
        'device':torch.device("cuda:1" if torch.cuda.is_available() else "cpu"),
        # 'device':torch.device("cpu"),
        # 'dataset':'CIFAR10',
        # 'dataset':'bank',
        # 'dataset':'credit',
        # 'dataset':'purchase',
        'dataset':'Iris',
        # 'result_dir': '20240702-FIL/',
        'result_dir': '20240702-effectiveInfo/',
        'oneData_bs': 30,
        'test_bs': 1,
        'train_bs':1,
        'noise_scale':0, # 防护措施
        'OneData':True,
        'split_layer': 3,
        # 'test_num': 'invdFIL', # MI, invdFIL, distCor, ULoss, 
        'test_num': 'effectiveInfo'
        }
print(args['device'])


cuda:1


In [27]:
data_msg = get_dataloader(args)
model_msg = get_models(args)
msg = {**model_msg,**data_msg}

# 数据集
one_data_loader,trainloader,testloader = data_msg['one_data_loader'],data_msg['trainloader'], data_msg['testloader']

# 模型和路径
client_net,decoder_net = model_msg['client_net'],model_msg['decoder_net']
decoder_route = model_msg['decoder_route']
image_deprocess = model_msg['image_deprocess']

results_dir = model_msg['results_dir']
inverse_dir = results_dir + 'layer'+str(args['split_layer'])+'/'
data_type = 1 if args['dataset'] == 'CIFAR10' else 0
split_layer = args['split_layer']

print('results_dir:',results_dir)
print('inverse_dir:',inverse_dir)
print('decoder_route:',decoder_route)

create_dir(results_dir)

# client_net使用
client_net = client_net.to(args['device'])
client_net.eval()


===============processing data===============
X_train.shape: (120, 4)
X_test.shape: (30, 4)
y_train.shape: (120,)
y_test.shape: (30,) <class 'numpy.ndarray'>
===============processing data end===============


linear1.weight
linear1.bias
linear2.weight
linear2.bias
train decoder model...
['Tanh', ('D', 128, 64), 'Tanh', ('D', 4, 128)]
results_dir: ../../results/20240702-effectiveInfo//Iris/effectiveInfo/
inverse_dir: ../../results/20240702-effectiveInfo//Iris/effectiveInfo/layer3/
decoder_route: ../../results/20240702-effectiveInfo//Iris/effectiveInfo//Decoder-layer3.pth


IrisNet(
  (linear1): Linear(in_features=4, out_features=128, bias=True)
  (linear2): Linear(in_features=128, out_features=64, bias=True)
)

# 2. pytorch的自动梯度

In [4]:
# 前向传播
import torch

x = torch.ones(5, requires_grad=True)  # input tensor
y = torch.zeros(3, requires_grad=True)  # expected output
w = torch.randn(5, 3, requires_grad=True)
b = torch.randn(3, requires_grad=True)
z = torch.matmul(x, w)+b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

# 信息打印
print(x.shape)
print(x.requires_grad)
print(z.shape)

torch.Size([5])
True
torch.Size([3])


In [ ]:
# 反向传播
loss.backward()
print(x.grad.shape)

In [ ]:
# grad/backward 张量求梯度 sum
x = torch.ones(5, requires_grad=True)  # input tensor
y = torch.zeros(3, requires_grad=True)  # expected output
w = torch.randn(5, 3)
b = torch.randn(3)
z = torch.matmul(x, w)+b

# 计算sumed雅可比矩阵
z.backward(torch.ones_like(z))
print(x.grad.shape)
print(x.grad)

# 用grad
from torch.autograd import grad
x = torch.ones(5, requires_grad=True)  # input tensor
y = torch.zeros(3, requires_grad=True)  # expected output
w = torch.eye(5, 3)
b = torch.randn(3)
z = torch.matmul(x, w)+b
xgrad = grad(z, x, grad_outputs=torch.ones_like(z))[0]
print(xgrad)


In [ ]:
# jacobian
from torch.autograd.functional import jacobian
from torch.autograd import grad
x = torch.ones(5, requires_grad=True)  # input tensor

def forward(x):
    y = torch.zeros(3, requires_grad=True)  # expected output
    w = torch.eye(5, 3)
    b = torch.randn(3)
    z = torch.matmul(x, w)+b
    return z


# 计算sumed雅可比矩阵
jac = jacobian(forward, x)
print(jac.shape)
print(jac)
print(jac.sum(dim=0))

# 3. 试探FIL计算

In [ ]:
# FIL计算：
import torch.autograd.functional as F
# 切割模型通讯量查看
# for i in range(7):
vgg5 = VGG('Client', 'VGG5', 1, model_cfg)

client_outputs = vgg5(images)
print('outputs.shape:',client_outputs.shape)
jacs = F.jacobian(vgg5, images)
print('jacobian: ', jacs)
# print('output size:',torch.prod(torch.tensor(list(client_outputs.shape))[1:]))
print('output size:',torch.prod(torch.tensor(list(client_outputs.shape))))

# 0到6层每层的jacobians
import torch.autograd.functional as F
for i in range(7):
    vgg5 = VGG('Client', 'VGG5', i, model_cfg)
    client_outputs = vgg5(images)
    print('outputs.shape:',client_outputs.shape)
    jacs = F.jacobian(vgg5, images)
    print('jacobian: ', jacs)
    # print('output size:',torch.prod(torch.tensor(list(client_outputs.shape))[1:]))
    print('output size:',torch.prod(torch.tensor(list(client_outputs.shape))))



In [ ]:
# FIL 计算，摸索出来一条路
import torch.autograd.functional as F
# 参数：
sigma = 0.01

# 计算jacobian
# 取一个batch的数据
train_iter=iter(trainloader)
inputs,labels = train_iter.next()6
print("inputs.shape: ",inputs.shape)
print("labels.shape: ",labels.shape)
print(f"input.requires_grad: {inputs.requires_grad}")

# 加载模型：
# vgg5 = VGG('Client', 'VGG5', 1, model_cfg)

# 进行前向传播：
inputs.requires_grad_(True) # 需要求导
outputs = vgg5(inputs)
outputs = outputs + sigma * torch.randn_like(outputs) # 加噪声 (0,1] uniform
print("outputs.shape: ",outputs.shape)

# 1. 进行反向传播,计算jacobian
# outputs.backward(torch.ones_like(outputs))
# J = inputs.grad / sigma # 计算jacobian
# print(f"J1.shape: {J.shape}")

# 2. 重新计算jacobian（用torch.autograd.functional.jacobian函数）
J = F.jacobian(vgg5, inputs)
# print(f"J2.shape: {J.shape}, J2.prod: {torch.prod(torch.tensor(list(J.shape)))}")
J = J.reshape(J.shape[0],outputs.numel(),inputs.numel())
print(f"J2.shape: {J.shape}, J2.prod: {torch.prod(torch.tensor(list(J.shape)))}")

# 计算eta 源论文
# J = model.influence_jacobian(train_data)[:, :, :-1] / args.sigma  # 计算FIL（梯度）jacobian
# etas = J.pow(2).sum(1).mean(1).sqrt() # 计算dFIL(这时候不是spectral norm了) 

# 计算eta：drj摸索：
I = torch.matmul(J[0].t(), J[0])
dFIL = I.trace().div(inputs.numel())
eta = dFIL.sqrt()
print(f"eta: {eta}")


In [ ]:
# dFIL的两个要求: 可导 + unbiased
# x = torch.rand_like(torch.Tensor([1,5]))
x = torch.Tensor([0,0])
x.requires_grad_(True)
print(x.grad)
y = torch.nn.ReLU()
z = y(x).sum()
# z = torch.autograd.functional.jacobian(y, x)
z.backward()
print(x)
print(y)
print(z)
print(x.grad)

# 4. 现成函数调用

In [28]:
# 我实现的：
# dFIL inverse指标计算

eta_same_layer_list = []
eta_diff_layer_list=[]

metric = dFILInverseMetric()
# 对traingloader遍历计算所有 inverse dFIL
# for j, data in enumerate(tqdm.tqdm(testloader)):
for j, data in enumerate(tqdm.tqdm(one_data_loader)): # 测试第一个testloader
    # if j < 31705:
        # continue
    inputs, labels = data
    inputs, labels = inputs.to(args['device']), labels.to(args['device'])
    
    # inference
    outputs = client_net(inputs)

    eta = metric.quantify(model=client_net, inputs=inputs, outputs=outputs, with_outputs=True)
    # 打印
    # print(str(j)+": "+str(eta.item()))
    eta_same_layer_list.append(eta)
eta_diff_layer_list.append(eta_same_layer_list)

# 结果储存到csv中
matrix = np.array(eta_diff_layer_list) # 有点大
transpose = matrix.T # 一行一条数据，一列代表一个layer 
pd.DataFrame(data=transpose, columns=[split_layer]).to_csv(results_dir + f'inv_dFIL{split_layer}.csv',index=False)

print(matrix)

100%|██████████| 1/1 [00:00<00:00,  1.58it/s]

[[2.0611473e-05]]


In [29]:
def calc_tr(net, x, device, sigmas=0.01, subsample=-1, jvp_parallelism=1): # nips'23 源码
    '''
    calc_tr 函数利用雅可比向量积（JVP）来估计网络对于输入数据的迹，
    这在分析网络的灵敏度或稳定性时非常有用。
    此外，通过支持子采样和并行处理，该函数还提供了一种在保持计算效率的同时估计迹的方法。
    '''
    print(f'x.shape: {x.shape}')
    
    # 定义一个局部函数 jvp_func**：这个函数接受两个参数 x 和 tgt，并返回 net.forward_first 方法的雅可比向量积（JVP）。
    # 这意味着 jvp_func 用于计算网络对于输入 x 在方向 tgt 上的一阶导数
    # tgt 计算雅各比向量积的向量
    def jvp_func(x, tgt):
        # return jvp(net.forward_first, (x,), (tgt,)) #返回 outputs, jacobian product
        return jvp(net.forward, (x,), (tgt,)) #返回 outputs, jacobian product

    # 获取一个batch中第一个数据的维度？d代表的是批次中第一个数据点展平后的特征数量，即输入数据的维度。
    d = x[0].flatten().shape[0] # 把一个batch的x展平，获取input dim

    # 用于存储每个输入数据点的迹，求迹的和。
    tr = torch.zeros(x.shape[0], dtype=x.dtype).to(device)
    #print(f'd: {d}, {x.shape}')

    # 加速，矩阵降维，但是这个损伤精度，或许改成特征提取更好点？
    # Randomly subsample pixels for faster execution
    if subsample > 0:
        samples = random.sample(range(d), min(d, subsample))
    else:
        samples = range(d)

    #print(x.shape, d, samples)
    # jvp parallelism是数据并行的粒度？
    # 函数通过分批处理样本来计算迹，每批处理 jvp_parallelism 个样本
    for j in range(math.ceil(len(samples) / jvp_parallelism)): # 对于每个数据块
        tgts = []

        # 遍历每个数据块中的每个维度
        '''
        在这个函数中，tgt 是用于计算雅可比向量积（JVP）的向量。具体来说，tgt 的作用如下：
        构建雅可比向量积的向量：tgt 是一个与输入 x 形状相同的张量，但它的元素大部分为零，只有一个特定位置的元素为 1。这个特定位置对应于我们在计算迹时关注的特征维度。
        计算 JVP：在 helper 函数中，tgt 被传递给 jvp_func，用于计算网络对于输入 x 在方向 tgt 上的一阶导数。具体来说，jvp_func 计算的是网络输出相对于输入 x 的雅可比矩阵与 tgt 的乘积。
        估计迹：通过在不同的特征维度上重复上述过程，可以估计网络对于输入数据的迹。迹的计算涉及到对所有特征维度的导数进行求和，而 tgt 的作用就是在每次计算时只关注一个特征维度。
        简而言之，tgt 是一个用于选择特定特征维度的向量，通过它可以逐个计算每个特征维度的导数，从而最终估计整个输入数据的迹。
        '''
        for k in samples[j*jvp_parallelism:(j+1)*jvp_parallelism]: # 提取整个batch中每个数据的特定维度
            tgt = torch.zeros_like(x).reshape(x.shape[0], -1) # 按照batch 排列？# 雅各比向量积的
            # 除了当前样本索引 k 对应的元素设置为 1。这相当于在计算迹时，每次只关注一个特征维度。
            tgt[:, k] = 1. # 提取tgt所有的样本的k的特征 计算雅各比向量积的向量，可用于计算trace
            tgt = tgt.reshape(x.shape) # 又变回x的形状
            tgts.append(tgt)
        tgts = torch.stack(tgts)

        # 定义一个辅助函数 helper，该函数接受一个目标张量 tgt并返回一个迹的张量和一个值的张量。
        # jvp wrapper，遍历每个batchsize
        def helper(tgt):
            batch_size = x.shape[0]
            vals_list = []
            grads_list = []
            for i in range(batch_size):
                val, grad = jvp_func(x[i], tgt[i])  # 对每个批次元素调用jvp_func
                vals_list.append(val)
                grads_list.append(grad)
            # 将结果列表转换为张量, 多个batch的给stack起来
            vals = torch.stack(vals_list)
            grad = torch.stack(grads_list)


            # vals, grad = vmap(jvp_func, randomness='same')(x, tgt)
            #print('grad shape: ', grad.shape)
            # 因此，矩阵平方的迹和迹的平方通常是不相等的。
            # 先求平方再求迹
            return torch.sum(grad * grad, dim=tuple(range(1, len(grad.shape)))), vals 

        # vmap被替换，
        # 遍历每个数据块
        trs,vals = [],[]
        for item in tgts:
            trs_, vals_ = helper(item)
            trs.append(trs_) # 每个batch对应一个向量
            vals.append(vals_)
        trs,vals = torch.stack(trs),torch.stack(vals)
        print(trs)
        # trs, vals = vmap(helper, randomness='same')(tgts) # randomness for randomness control of dropout
        
        # vals are stacked results that are repeated by d (should be all the same)


        tr += trs.sum(dim=0) # 对每个数据块的迹求和

    # Scale if subsampled
    if subsample > 0:
        tr *= d / len(samples)

    # 1/dFIL = d/tr(I)
    tr = tr/(d*1.0)
    tr = 1.0/tr*sigmas

    # print('tr: ',tr.shape, tr)
    return tr.cpu().item(), vals[0].squeeze(1)  # squeeze removes one dimension jvp puts


In [30]:
# maeng fisher

eta_same_layer_list = []
eta_diff_layer_list=[]

metric_trace = dFILInverseMetric()
for j, data in enumerate(tqdm.tqdm(one_data_loader)): # 测试第一个testloader
    # if j < 31705:
        # continue
    inputs, labels = data
    inputs, labels = inputs.to(args['device']), labels.to(args['device'])
    
    # inference
    # outputs = client_net(inputs)

    inputs = inputs.unsqueeze(0)
    eta,val = metric_trace.calc_tr(net=client_net, x=inputs, device=args['device'])
    # 打印
    # print(str(j)+": "+str(eta.item()))
    eta_same_layer_list.append(eta)
eta_diff_layer_list.append(eta_same_layer_list)

# 结果储存到csv中
matrix = np.array(eta_diff_layer_list) # 有点大
print("matrix: ",matrix)
transpose = matrix.T # 一行一条数据，一列代表一个layer 
pd.DataFrame(data=transpose, columns=[split_layer]).to_csv(results_dir + f'inv_dFIL_maeng-{split_layer}.csv',index=False)

  0%|          | 0/1 [00:00<?, ?it/s]

x.shape: torch.Size([1, 30, 4])


100%|██████████| 1/1 [00:00<00:00,  3.64it/s]

matrix:  [[2.06114892e-05]]
